In [ ]:
"""
# For Google Colab training

!pip install pytorch_lightning pedalboard

from google.colab import drive
drive.mount('/content/drive')

from drive.MyDrive.apr.src.datasets import VoxCelebDataModule
from drive.MyDrive.apr.src.losses import (
    SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from drive.MyDrive.apr.src.models import (
    build_efficientnetv2,
    MHA_LAS, ResNet34SEV2
)

import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(42, workers=True)

!unzip "/content/drive/MyDrive/datasets/voxceleb1/subset4.zip" -d "/content/subset"
"""

In [ ]:
import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.losses import (
    SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from src.models import (
    build_efficientnetv2,
    MHA_LAS, ResNet34SEV2
)

seed_everything(42, workers=True)

In [ ]:
num_classes = 100
batch_size = 8
n_mels = 80
from_memory = False
spec_augment = False

dm2 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=2
)
dm4 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=4,
    clip_test=True
)

# ResNet34-SE

## Train (2 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/2secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15,
    num_subcenters=3
)
model = ResNet34SEV2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.001,
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="2secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm2
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.15,
    scale=20,
    num_subcenters=3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SEV2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.001
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    ckpt_path="checkpoints/resnet34se/2secs/epoch=18-step=17574.ckpt"
)

## Test (4 secs)

In [ ]:
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.15,
    scale=20,
    num_subcenters=3
)
model = ResNet34SEV2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.001,
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    num_sanity_val_steps=0
)
trainer.test(
    model, 
    dm4,
    ckpt_path="checkpoints/resnet34se/4secs/epoch=23-step=22199.ckpt"
)

# EfficientNetV2

## Train (2 secs)

In [ ]:
checkpoint_dir = "/content/drive/MyDrive/checkpoints/"
logs_dir = "/content/drive/MyDrive/tb_logs"

checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir + "efficientnetv2/2secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam",
)
logger = TensorBoardLogger(
    logs_dir, 
    name="efficientnetv2",
    sub_dir="2secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm2,
    ckpt_path=checkpoint_dir + "efficientnetv2/2secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/efficientnetv2/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam"
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="efficientnetv2",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    ckpt_path="checkpoints/efficientnetv2/4secs/last.ckpt"
)

## Test (4 secs)

In [ ]:
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="efficientnetv2",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    num_sanity_val_steps=0
)
trainer.test(
    model, 
    dm4,
    ckpt_path="checkpoints/efficientnetv2/4secs/epoch=69-step=82305.ckpt"
)

# LAS-MHA

## Train (2 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/2secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="2secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm2,
    ckpt_path="checkpoints/MHA_LAS/2secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.15,
    scale=20,
    num_subcenters=3
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    ckpt_path="checkpoints/MHA_LAS/4secs/epoch=28-step=26824.ckpt"
)

## Test (4 secs)

In [ ]:
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.15,
    scale=20,
    num_subcenters=3
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    num_sanity_val_steps=0
)
trainer.test(
    model, 
    dm4,
    ckpt_path="checkpoints/MHA_LAS/4secs/epoch=28-step=26824.ckpt"
)